# [실습4] Chathistory를 가진 챗봇과 질의응답

## 실습 목표
---
[실습4] 챗봇의 기능 중 Chat History를 기억하지 못하는 문제를 LangChain을 통해 해결하기 위한 방법을 이해합니다.


## 실습 목차
---

1. **Chat History 저장 및 입력:** Chat History를 저장하고 적용하는 기능을 구현합니다.

## 실습 개요
---
본격적으로 챗봇의 기능을 고도화 하기 전, 챗봇의 퀄리티를 높일 수 있는 다양한 방법을 학습합니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [2]:
import os
import time

from langchain_community.chat_models import ChatOllama, ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

- OpenAI를 통해 gpt-4o-mini 모델을 불러옵니다.

gpt-4o-mini 모델을 사용하는 ChatOpenAI 객체를 생성합니다.

In [3]:
import os


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "A"

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
route_llm = ChatOpenAI(model="gpt-4o-mini", format="json")

간단한 질의응답 Chain을 구성합니다.

In [5]:
messages_with_variables = [
    ("system", "당신은 친절한 교통 분야 전문 AI Assistant 입니다."),
    ("human", "{question}"),
]
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages(messages_with_variables)
chain = prompt | llm | parser

## 1. Chat History 저장 및 적용

챗봇은 기본적으로 이전 대화 내용을 기억하지 않습니다. 즉, 유저가 자신의 이름을 말하거나 이전 질문에 이어지는 질문을 해도 챗봇은 이를 기억하지 못하고 대화를 이해할 수 있는 능력이 떨어집니다.

저희가 프로젝트에서 구현하는 챗봇은 이러한 기억 능력이 없어도 필요한 정보를 충분히 Retrieve 할 수 있지만, Chat History를 기억해야 하는 다른 챗봇을 구현할 때는 문제가 될 수 있습니다.

LangChain은 이러한 Chat History를 저장하는 메모리와 관련된 다양한 모듈들을 지원합니다. 이번 실습에서는 LangChain에서 제공하는 다양한 메모리 모듈을 활용하여, LLM이 대화 내용을 기억할 수 있도록 해보겠습니다.

우선 아래의 셀을 실행해보겠습니다.

In [ ]:
response = chain.invoke("챗봇을 만드는 순서를 한국어로 설명해줘")

print(response)

In [ ]:
response = chain.invoke(
    "방금 말한 것을 영어로 번역해줘"
)
print(response)

간단한 질문에 잘 대답하는 것을 볼 수 있지만, 답변한 내용에 기반해서 다시 물어보면 뜬금없는 얘기를 하는 것을 볼 수 있습니다. 이는 LangChain에서 LLM은 기본적으로 응답을 하면, 기존 대화 기록을 저장하지 않기 때문입니다.


이번 실습에서는 LangChain에서 제공되는 다양한 Memory 모듈을 활용하여, LLM이 대화 내용을 기억할 수 있도록 해보겠습니다.

## 1-1 ConversationBufferMemory

단순히 이전 대화 내용 전체를 저장하는 메모리입니다


`ConversationBufferMemory`는 단순히 이전 대화 내용 전체를 저장하는 메모리입니다. 이를 사용하여 대화의 맥락을 유지할 수 있습니다.

__메모리 설정하기__


우선 `ConversationBufferMemory` 객체를 `memory` 변수에 할당하여, `memory에` 이전 LLM과의 대화 내용을 계속 저장하겠습니다.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

memory.load_memory_variables({})

memory는 `load_memory_variables` 메소드를 사용해서 이전 대화 내용을 확인할 수 있습니다. 기본적으로 처음 memory를 만들면 비어있는 것을 확인할 수 있습니다.

__대화 내용 저장하기__


`save_context` 메소드를 활용하여 LLM과의 대화 내용을 추가해줄 수 있습니다. `save_context` 활용 시 "input"에는 사용자의 입력, "output"에는 LLM의 응답을 기록해줍니다.


In [ ]:
memory.save_context(
    {"input": "나는 국토교통부에서 근무하고 싶은 김싸피야."}, {"output": "꿈을 이루길 바래요!"}
)

memory.load_memory_variables({})

### [TODO] memory 변수를 만들고 체인을 구성해보세요.

In [ ]:
memory = ____________________(
    memory_key="chat_history",
    return_messages=True,
)


def load_memory(x):
    return memory.load_memory_variables({})["chat_history"]


messages_with_history = [
    ("system", "당신은 친절한 교통 분야 AI Assistant 입니다."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
]
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages(messages_with_history)
memory_chain = (
    {"chat_history": load_memory, "question": RunnablePassthrough()}
    | __________
    | __________
    | __________
)

In [ ]:
question = "챗봇을 만드는 순서를 설명해줘"
response = memory_chain.invoke(question)

print(response)

memory.save_context({"input": question}, {"output": response})

In [ ]:
response = memory_chain.invoke(
    "방금 말한 것을 한글로 번역해줘"
)
print(response)

In [ ]:
memory

이제 메모리가 추가된 체인을 통해, 이전 대화에 대한 정보를 포함한 응답을 얻을 수 있습니다. 메모리가 없을 때는 이전 대화에 대해 물어보면 이상한 말을 하는 것을 볼 수 있었는데, 메모리를 추가하면 제대로 응답하는 모습을 확인할 수 있습니다.

하지만, `ConversationBufferMemory`는 몇 가지 한계점이 있습니다:

- 비용 증가: 모델 자체에는 메모리가 없으므로, 모델을 호출할 때마다 전체 대화 기록을 프롬프트에 함께 보내야 합니다. 이는 비용을 증가시킵니다.
- 집중 분산: 언어 모델이 참조하는 이전의 텍스트가 너무 많아지면, 언어모델은 중요한 부분을 놓칠 수 있습니다.

In [ ]:
question = "How can I safely merge onto the highway?"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "Summarize What you just said in Korean"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "How can I avoid sudden stops while driving in heavy traffic?"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "Summarize What you just said in Korean"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "What are the best practices for safe driving in rainy conditions?"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "Summarize What you just said in Korean"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

print(memory.load_memory_variables({}))


모든 대화가 계속해서 메모리에 저장되는 모습을 볼 수 있습니다. 이는 대화 내용이 길어질수록 LLM에게 물어보는 비용이 증가하게 되고, 중요한 부분을 놓치게 될 가능성도 커집니다.

## 1-2 ConversationBufferWindowMemory

`ConversationBufferWindowMemory`는 대화의 특정 부분만을 저장하는 메모리입니다. 예를 들어, 최근 5개의 대화만 저장하는 식입니다.


- 모든 대화를 저장하지 않아도 되어 메모리 사용량이 줄어듭니다.
- 하지만 챗봇이 최근 대화에만 집중하게 되며, 이전 대화를 기억하지 못합니다.

### [TODO] memory 변수를 만들고 체인을 구성해보세요.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ____________________(
    memory_key="chat_history",
    return_messages=True,
    k=4,
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def load_memory(x):
    return memory.load_memory_variables({})["chat_history"]


messages_with_history = [
    ("system", "당신은 친절한 교통 분야 전문 AI Assistant 입니다."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
]
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages(messages_with_history)
memory_chain = (
    {"chat_history": load_memory, "question": RunnablePassthrough()}
    | __________
    | __________
    | __________
)

question = "How can I safely merge onto the highway?"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "Summarize What you just said in Korean"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "How can I avoid sudden stops while driving in heavy traffic?"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "Summarize What you just said in Korean"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "What are the best practices for safe driving in rainy conditions?"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "Summarize What you just said in Korean"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

print(memory.load_memory_variables({}))


위의 코드를 실행하고 모델의 메모리를 살펴보면, 최근 4번의 대화 기록만 남아있는 것을 확인할 수 있습니다.

대화 내용이 길어질 때 메모리 사용량을 줄이고 비용을 절감할 수 있습니다. 하지만 이 방법의 단점은 챗봇이 과거에 나눈 대화를 기억하지 못하게 되는 것입니다.

## 1-3 ConversationSummaryBufferMemory

`ConversationSummaryBufferMemory`는 대화 내용이 길어질 때, 오래된 메시지를 요약하여 메모리에 저장하는 기능을 제공합니다. 이는 오래된 메시지를 단순히 삭제하는 대신 요약하여 보관함으로써, 모델이 가장 최근의 상호작용에 집중하면서도, 이전 대화 내용도 잊지 않도록 합니다.

- 최근 대화 내용에 집중하면서도, 오래된 대화 내용은 요약하여 유지합니다.
- 대화가 길어져도 메모리 사용량을 효과적으로 관리할 수 있습니다.


__작동 방식:__

1. 메시지 수가 한계에 도달: 메모리에 저장된 메시지의 수가 설정된 한계에 도달하면,
2. 오래된 메시지 요약: 오래된 메시지들을 LLM을 호출하여 요약합니다.
3. 요약된 메시지 저장: 요약된 메시지를 메모리에 저장하여, 중요한 내용을 잊지 않도록 합니다.

### [TODO] memory 변수를 만들고 체인을 구성해보세요.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ____________________(
    memory_key="chat_history",
    llm=llm,
    max_token_limit=400,
    return_messages=True
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def load_memory(x):
    return memory.load_memory_variables({})["chat_history"]

messages_with_history = [
    ("system", "당신은 친절한 교통 분야 전문 AI Assistant 입니다."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
]
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages(messages_with_history)
memory_chain = (
    {"chat_history": load_memory, "question": RunnablePassthrough()}
    | __________
    | __________
    | __________
)

question = "How can I safely merge onto the highway?"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "Summarize What you just said in Korean"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "How can I avoid sudden stops while driving in heavy traffic?"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "Summarize What you just said in Korean"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "What are the best practices for safe driving in rainy conditions?"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

question = "Summarize What you just said in Korean"
response = memory_chain.invoke(question)
memory.save_context({"input": question}, {"output": response})

print(memory.load_memory_variables({}))


위의 코드를 실행하면, ConversationSummaryBufferMemory가 설정한 최대 토큰 한계(400)를 넘어가면서 오래된 대화 내용이 요약된 것을 확인할 수 있습니다. 이를 통해 가장 최근의 대화 내용은 그대로 유지하면서, 오래된 대화 내용도 요약된 형태로 잊혀지지 않고 보존할 수 있습니다.